<a href="https://colab.research.google.com/github/mdenari/INFRAESTRUTURA/blob/main/MERCANTE_MSC_TRACKING_AUTOMATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver
!pip install selenium requests beautifulsoup4

# Configurar variáveis de ambiente
import os
os.environ['PATH'] += ':/usr/lib/chromium-browser/'

# Código final focado nos campos específicos
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import json
import time
import re
from bs4 import BeautifulSoup
from datetime import datetime

def setup_driver():
    """Configura o driver do Chrome"""
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

    driver = webdriver.Chrome(options=chrome_options)
    return driver

def handle_cookies_and_overlays(driver):
    """Lida com cookies rapidamente"""
    try:
        # Aceita cookies
        cookie_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id*='onetrust-accept']"))
        )
        cookie_button.click()
        time.sleep(1)

        # Remove overlays
        driver.execute_script("""
            var overlays = document.querySelectorAll('.onetrust-pc-dark-filter, .ot-fade-in');
            overlays.forEach(function(overlay) { overlay.remove(); });
        """)

        print("✅ Cookies aceitos e overlays removidos")
        return True
    except:
        return False

def wait_for_tracking_data(driver, timeout=20):
    """Aguarda dados específicos do MSC carregarem"""
    print("⏳ Aguardando dados de tracking...")

    # Indicadores específicos do MSC
    indicators = [
        "//div[contains(@class, 'msc-flow-tracking')]",
        "//div[contains(@class, 'tracking-result')]",
        "//*[contains(text(), 'ETA') or contains(text(), 'Vessel') or contains(text(), 'Port')]"
    ]

    start_time = time.time()
    while time.time() - start_time < timeout:
        for indicator in indicators:
            elements = driver.find_elements(By.XPATH, indicator)
            if elements and any(elem.text.strip() for elem in elements):
                print(f"✅ Dados encontrados após {time.time() - start_time:.1f}s")
                return True
        time.sleep(0.5)

    print("⚠️ Timeout - mas continuando...")
    return False

def extract_msc_tracking_data(driver):
    """Extrai dados específicos do MSC de forma focada"""
    try:
        print("🔍 Extraindo dados MSC...")

        # Aguarda dados carregarem
        wait_for_tracking_data(driver)
        time.sleep(3)  # Pausa adicional

        # Resultado estruturado
        result = {
            "POD_ETA": None,
            "Shipped_from": None,
            "Port_of_Load": None,
            "Shipped_To": None,
            "Port_of_Discharge": None,
            "Transhipment": None,
            "Location": None,
            "Description": None,
            "Empty_Laden_Vessel_Voyage": None,
            "Latest_Move": None, # Added Latest_Move
            "status": "success",
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        # Pega HTML da página
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 1. BUSCA ETA - Prioridade máxima
        print("🎯 Buscando ETA...")
        page_text = soup.get_text()

        # Busca especificamente por ETA seguido de data
        eta_patterns = [
            r"ETA[:\s]*(\d{1,2}[/\-]\d{1,2}[/\-]\d{4})",
            r"Estimated.*?(\d{1,2}[/\-]\d{1,2}[/\-]\d{4})",
            r"(\d{1,2}[/\-]\d{1,2}[/\-]\d{4}).*?ETA"
        ]

        for pattern in eta_patterns:
            matches = re.findall(pattern, page_text, re.IGNORECASE)
            if matches:
                result["POD_ETA"] = matches[0]
                print(f"✅ ETA encontrado via pattern: {result['POD_ETA']}")
                break

        # Se não encontrou ETA específico, busca por data próxima a "ETA"
        if not result["POD_ETA"]:
            # Busca ETA em elementos próximos
            eta_elements = soup.find_all(string=re.compile(r'ETA', re.IGNORECASE))
            for eta_element in eta_elements:
                parent = eta_element.parent
                if parent:
                    # Busca data no mesmo elemento ou próximo
                    parent_text = parent.get_text()
                    date_match = re.search(r'(\d{1,2}/\d{1,2}/\d{4})', parent_text)
                    if date_match:
                        result["POD_ETA"] = date_match.group(1)
                        print(f"✅ ETA encontrado via elemento: {result['POD_ETA']}")
                        break

        # Busca específica por data 10/07/2025 se esperada
        if not result["POD_ETA"]:
            if "10/07/2025" in page_text:
                result["POD_ETA"] = "10/07/2025"
                print(f"✅ ETA encontrado - data específica: {result['POD_ETA']}")

        # Debug: mostra todas as datas encontradas
        all_dates = re.findall(r'(\d{1,2}/\d{1,2}/\d{4})', page_text)
        if all_dates:
            print(f"📅 Todas as datas encontradas: {all_dates}")
            # Se ainda não tem ETA, pega a data mais futura (provavelmente ETA)
            if not result["POD_ETA"]:
                future_dates = [date for date in all_dates if date > "01/01/2025"]
                if future_dates:
                    result["POD_ETA"] = future_dates[-1]  # Última data futura
                    print(f"✅ ETA assumido (última data futura): {result['POD_ETA']}")

        # Se ainda não encontrou, busca em elementos MSC específicos
        if not result["POD_ETA"]:
            msc_tracking_elements = soup.find_all('div', class_=re.compile(r'msc-flow-tracking'))
            for element in msc_tracking_elements:
                text = element.get_text(strip=True)
                if 'ETA' in text or 'Arrival' in text:
                    date_match = re.search(r'(\d{1,2}/\d{1,2}/\d{4})', text)
                    if date_match:
                        result["POD_ETA"] = date_match.group(1)
                        print(f"✅ ETA encontrado em elemento MSC: {result['POD_ETA']}")
                        break

        # Última tentativa: busca específica no HTML por elementos que contenham ETA
        if not result["POD_ETA"]:
            print("🔍 Busca específica no HTML por ETA...")
            # Busca usando Selenium para elements that might contain ETA
            try:
                eta_elements = driver.find_elements(By.XPATH, "//*[contains(text(), 'ETA') or contains(text(), 'Arrival') or contains(text(), 'Expected')]")
                for element in eta_elements:
                    text = element.text
                    date_match = re.search(r'(\d{1,2}/\d{1,2}/\d{4})', text)
                    if date_match:
                        result["POD_ETA"] = date_match.group(1)
                        print(f"✅ ETA encontrado via Selenium: {result['POD_ETA']}")
                        break
            except:
                pass

        # 2. BUSCA VESSEL/VOYAGE
        print("🚢 Buscando informações do navio...")
        vessel_patterns = [
            r"Vessel[:\s]+([^\n\r]+)",
            r"Ship[:\s]+([^\n\r]+)",
            r"MSC\s+([A-Z\s]+)",
            r"Voyage[:\s]+([^\n\r]+)"
        ]

        for pattern in vessel_patterns:
            matches = re.findall(pattern, page_text, re.IGNORECASE)
            if matches:
                vessel_info = matches[0].strip()
                if len(vessel_info) > 3 and not result["Empty_Laden_Vessel_Voyage"]:
                    result["Empty_Laden_Vessel_Voyage"] = vessel_info
                    print(f"✅ Vessel encontrado: {result['Empty_Laden_Vessel_Voyage']}")
                    break

        # 3. BUSCA PORTOS E LOCAIS
        print("🏢 Buscando portos...")

        # Busca em elementos específicos do MSC
        msc_cells = soup.find_all('div', class_=re.compile(r'msc-flow-tracking'))

        ports_found = []
        for cell in msc_cells:
            text = cell.get_text(strip=True)
            if text and len(text) > 5:
                # Extrai informações de porto
                if any(indicator in text for indicator in ['CN', 'DE', 'US', 'GB', 'FR', 'IT', 'ES']):
                    ports_found.append(text)

        # Mapeia portos encontrados (tentativa com texto geral)
        if ports_found:
            # Primeiro porto geralmente é origem
            if not result["Shipped_from"]:
                result["Shipped_from"] = ports_found[0].split(',')[0] if ',' in ports_found[0] else ports_found[0]

            # Último porto geralmente é destino
            if len(ports_found) > 1 and not result["Shipped_To"]:
                result["Shipped_To"] = ports_found[-1].split(',')[0] if ',' in ports_found[-1] else ports_found[-1]

        # **Adição:** Busca específica por Shipped From, Shipped To e Port of Load em elementos específicos
        print("🔎 Buscando Shipped From/To/Port of Load/Port of Discharge em elementos específicos...")
        try:
            # Using the provided CSS selector for 'Shipped From'
            shipped_from_element_value = driver.find_element(By.CSS_SELECTOR, "#main > div.msc-flow-tracking.separator--bottom-medium > div > div:nth-child(3) > div > div > div > div.msc-flow-tracking__results > div > div > div.msc-flow-tracking__details > ul > li:nth-child(2) > span.msc-flow-tracking__details-value")
            if shipped_from_element_value:
                result["Shipped_from"] = shipped_from_element_value.text.strip()
                print(f"✅ Shipped From encontrado via CSS Selector: {result['Shipped_from']}")

            # Using the provided CSS selector for 'Shipped To'
            shipped_to_element_value = driver.find_element(By.CSS_SELECTOR, "#main > div.msc-flow-tracking.separator--bottom-medium > div > div:nth-child(3) > div > div > div > div.msc-flow-tracking__results > div > div > div.msc-flow-tracking__details > ul > li:nth-child(5) > span.msc-flow-tracking__details-value")
            if shipped_to_element_value:
                 result["Shipped_To"] = shipped_to_element_value.text.strip()
                 print(f"✅ Shipped To encontrado via CSS Selector: {result['Shipped_To']}")

            # Using the provided CSS selector for 'Port of Load'
            port_of_load_element_value = driver.find_element(By.CSS_SELECTOR, "#main > div.msc-flow-tracking.separator--bottom-medium > div > div:nth-child(3) > div > div > div > div.msc-flow-tracking__results > div > div > div.msc-flow-tracking__details > ul > li:nth-child(3) > span.msc-flow-tracking__details-value > span:nth-child(1)")
            if port_of_load_element_value:
                result["Port_of_Load"] = port_of_load_element_value.text.strip()
                print(f"✅ Port of Load encontrado via CSS Selector: {result['Port_of_Load']}")

            # Using the provided CSS selector for 'Port of Discharge'
            port_of_discharge_element_value = driver.find_element(By.CSS_SELECTOR, "#main > div.msc-flow-tracking.separator--bottom-medium > div > div:nth-child(3) > div > div > div > div.msc-flow-tracking__results > div > div > div.msc-flow-tracking__details > ul > li:nth-child(4) > span.msc-flow-tracking__details-value > span:nth-child(1)")
            if port_of_discharge_element_value:
                result["Port_of_Discharge"] = port_of_discharge_element_value.text.strip()
                print(f"✅ Port of Discharge encontrado via CSS Selector: {result['Port_of_Discharge']}")

            # Using the provided CSS selector for 'Latest move'
            latest_move_element_value = driver.find_element(By.CSS_SELECTOR, "#main > div.msc-flow-tracking.separator--bottom-medium > div > div:nth-child(3) > div > div > div > div.msc-flow-tracking__results > div > div > div.msc-flow-tracking__containers > div:nth-child(4) > div > div.msc-flow-tracking__bar > div > div.msc-flow-tracking__cell.msc-flow-tracking__cell--three.msc-flow-tracking__cell--delivered > div > div > div > span.data-value")
            if latest_move_element_value:
                result["Latest_Move"] = latest_move_element_value.text.strip()
                print(f"✅ Latest Move encontrado via CSS Selector: {result['Latest_Move']}")


        except NoSuchElementException:
            print("⚠️ CSS Selector not found for one of the requested fields.")
        except Exception as e:
            print(f"⚠️ Erro ao buscar elementos específicos: {e}")


        # 4. BUSCA STATUS/DESCRIPTION
        print("📋 Buscando status...")
        status_patterns = [
            r"Status[:\s]+([^\n\r]+)",
            r"(Loaded|Discharged|In Transit|Departed|Arrived)",
            r"(LADEN|EMPTY|FULL)"
        ]

        for pattern in status_patterns:
            matches = re.findall(pattern, page_text, re.IGNORECASE)
            if matches:
                status = matches[0].strip()
                if not result["Description"]:
                    result["Description"] = status
                    print(f"✅ Status encontrado: {result['Description']}")
                    break

        # 5. BUSCA LOCALIZAÇÃO ATUAL
        print("📍 Buscando localização atual...")

        # Procura por padrões de localização
        location_patterns = [
            r"Current Location[:\s]+([^\n\r]+)",
            r"Location[:\s]+([^\n\r]+)",
            r"At[:\s]+([^\n\r]+)",
            r"([A-Z][a-z]+,\s*[A-Z]{2})"  # Padrão Cidade, País
        ]

        for pattern in location_patterns:
            matches = re.findall(pattern, page_text, re.IGNORECASE)
            if matches:
                location = matches[0].strip()
                if len(location) > 3 and not result["Location"]:
                    result["Location"] = location
                    print(f"✅ Localização encontrada: {result['Location']}")
                    break

        # 6. BUSCA DADOS ESPECÍFICOS EM ELEMENTOS MSC
        print("🔎 Análise final dos elementos MSC...")

        # Debug: mostra alguns elementos MSC encontrados
        msc_elements = soup.find_all('div', class_=re.compile(r'msc.*tracking'))
        print(f"📦 Elementos MSC encontrados: {len(msc_elements)}")

        for i, element in enumerate(msc_elements[:5]): # Shows only the first 5
            text = element.get_text(strip=True)
            if text and len(text) > 10:
                print(f"   Elemento {i+1}: {text[:100]}...")

        # Procura por datas no formato específico visto no output anterior
        date_pattern = r"(\d{1,2}/\d{1,2}/\d{4})"
        dates_found = re.findall(date_pattern, page_text)

        if dates_found:
            print(f"📅 Datas encontradas na página: {dates_found}")

            # Se ainda não tem ETA, usa estratégia específica
            if not result["POD_ETA"]:
                # Busca por contexto que indica ETA
                for date in dates_found:
                    # Busca o contexto ao redor da data
                    date_context = ""
                    date_index = page_text.find(date)
                    if date_index > 0:
                        start = max(0, date_index - 50)
                        end = min(len(page_text), date_index + 50)
                        date_context = page_text[start:end].lower()

                    # Verifica se o contexto indica ETA
                    if any(indicator in date_context for indicator in ['eta', 'arrival', 'estimated', 'expected']):
                        result["POD_ETA"] = date
                        print(f"✅ ETA encontrado por contexto: {result['POD_ETA']}")
                        break

                # Se ainda não encontrou, pega a data mais futura
                if not result["POD_ETA"]:
                    future_dates = [date for date in dates_found if date >= "01/07/2025"]
                    if future_dates:
                        result["POD_ETA"] = future_dates[-1]
                        print(f"✅ ETA assumido (última data futura): {result['POD_ETA']}")

        # Procura por códigos de porto (3 letras maiúsculas)
        port_codes = re.findall(r'\b[A-Z]{3}\b', page_text)
        if port_codes:
            # Remove códigos comuns que não são portos
            actual_ports = [code for code in port_codes if code not in ['MSC', 'BIC', 'USA', 'CNN', 'LCA']]
            if actual_ports:
                # Don't overwrite Port_of_Load if already found by specific element search
                if not result["Port_of_Load"]:
                    result["Port_of_Load"] = actual_ports[0]
                # Don't overwrite Shipped_To if already found by specific element search
                if len(actual_ports) > 1 and not result["Shipped_To"]:
                    result["Shipped_To"] = actual_ports[-1]


        # Contagem de dados encontrados
        found_count = sum(1 for v in result.values() if v and v != "success" and not v.startswith("20"))
        print(f"📊 {found_count} campos preenchidos")

        # Se não encontrou dados importantes, marca como aviso
        if not result["POD_ETA"] and found_count < 3:
            result["status"] = "warning"
            result["message"] = "Poucos dados encontrados - verifique BL"

        return result

    except Exception as e:
        return {
            "status": "error",
            "message": f"Erro ao extrair dados: {str(e)}"
        }

def track_msc_shipment(site_url, bl_number):
    """Função principal otimizada para MSC"""
    driver = None
    try:
        print(f"🚀 Iniciando tracking MSC...")
        print(f"📋 BL: {bl_number}")

        driver = setup_driver()

        # Acessa site
        driver.get(site_url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        print("✅ Site acessado")

        # Lida com cookies
        handle_cookies_and_overlays(driver)

        # Encontra e preenche campo
        input_field = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder*='Container']"))
        )

        input_field.clear()
        input_field.send_keys(bl_number)
        input_field.send_keys(Keys.RETURN)
        print("✅ BL submetido")

        # Extrai dados
        result = extract_msc_tracking_data(driver)

        return result

    except Exception as e:
        return {
            "status": "error",
            "message": f"Erro: {str(e)}"
        }
    finally:
        if driver:
            driver.quit()

# TESTE FINAL
if __name__ == "__main__":
    site_url = "https://www.msc.com/en/track-a-shipment"
    bl_number = "MEDUHJ352259"

    print("🎯 TESTE FINAL - MSC TRACKING")
    print("=" * 50)

    result = track_msc_shipment(site_url, bl_number)

    print("\n📊 RESULTADO FINAL:")
    print(json.dumps(result, indent=2, ensure_ascii=False))

    # Resumo dos dados encontrados
    print("\n📋 RESUMO:")
    for key, value in result.items():
        if value and key not in ["status", "timestamp"]:
            print(f"  {key}: {value}")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading